In [1]:
# CREATE TABLE items (id bigserial PRIMARY KEY, content TEXT, embedding vector(1536));

In [2]:
import os
from dotenv import find_dotenv, load_dotenv
from openai import OpenAI
import openai
import psycopg2

In [3]:
load_dotenv(find_dotenv())

True

In [4]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# Set up your OpenAI API key
# openai.api_key = OPENAI_API_KEY
client = OpenAI()
# Choose a model
model = "text-embedding-ada-002"
print(OPENAI_API_KEY[:12])

sk-proj-IMmV


In [5]:
print("Connecting to PostgreSQL...")
conn = psycopg2.connect(
    database="postgres",
    user="postgres",
    password="postgres",
    host="host.docker.internal",
)

cur = conn.cursor()

print("Successfully connected to PostgreSQL.")

Connecting to PostgreSQL...
Successfully connected to PostgreSQL.


In [6]:
# Converting the prompt to the pgvector embedding
def get_embedding(prompt):
    response = openai.embeddings.create(input=prompt, model="text-embedding-ada-002")

    embedding = response.data[0].embedding

    # Converting the embedding to the pgvector and returning it
    return "[" + ",".join(map(str, embedding)) + "]"

Second, find the most relevant movies for a provided user prompt by calculating the cosine distance (`<=>`) between the prompt's and movies' embeddings:


In [7]:
try:


    query = "Tell me about Inhibitor concentration"


    query_embedding = get_embedding(query)


    # Perform a cosine similarity search

    cur.execute(
        """

        SELECT id, element_text, 1 - (embedding <=> %s) AS cosine_similarity

            FROM tbl_doc_elements

            ORDER BY cosine_similarity DESC LIMIT 3
        """,

        (query_embedding,),
    )


    # Fetch and print the result

    print("Query:", query)

    print("Most similar sentences:")

    for row in cur.fetchall():

        print(f"ID: {row[0]},  Cos: {row[2]}, CONTENT: {row[1]},")


except Exception as e:

    print("Error executing query", str(e))

Query: Tell me about Inhibitor concentration
Most similar sentences:
ID: 541a665d-a5c5-4253-9113-0e391ded2215,  Cos: 0.04010286779744443, CONTENT: 15 - & — 10g =} — & —6g N Potential (V) — 2 & = Control 2 e 88 25 T T 0.0000001 0.00001 0.001 01 Current Density (A/cm2),
ID: ff3a5f52-18b4-4aec-b460-9ffe91816065,  Cos: 0.035088884987433455, CONTENT: 12   C/0  10  8  0  /  C  6  4  2  2  4  6  8  10  Concentration (g) ,
ID: 3d5560fe-ea3c-4135-bb3a-fea5da537cbc,  Cos: 0.027756672352552636, CONTENT: Inhibitor Polarization Corrosion be (V/dec) ba (V/dec) Ecorr (V) icorr (AJcm?) concentration (g) resistance (Q) rate (mmj/year) 0.0335 0.0409 —0.9393 0.0003 24.0910 2.8163 1.9460 0.0596 .8276 0.0002 121.440 1.5054 0.0163 0.2369 .8825 0.0001 42121 0.9476 s NO 03233 0.0540 —0.8027 5.39E-05 373.180 0.4318 0.1240 0.0556 .5896 5.46E-05 305.650 0.3772 = 5 0.0382 0.0086 .5356 1.24E-05 246.080 0.0919,


In [8]:
cur.close()
conn.close()